In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn

class HitModel(nn.Module):
    def __init__(self, feature_dim, num_consecutive_frames):
        super(HitModel, self).__init__()
        self.num_consecutive_frames = num_consecutive_frames
        self.feature_dim = feature_dim

        self.gru1 = nn.GRU(feature_dim // num_consecutive_frames, 64, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(128, 64, bidirectional=True, batch_first=True)
        self.global_maxpool = nn.MaxPool1d(num_consecutive_frames)
        self.dense = nn.Linear(128, 3)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size = x.shape[0]
        x=x.float()
        x = x.view(batch_size, self.num_consecutive_frames, self.feature_dim // self.num_consecutive_frames)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = x.transpose(1, 2)
        x = self.global_maxpool(x).squeeze()
        x= self.dense(x)
        x=self.softmax(x)
        return x

In [ ]:
import pandas as pd
import numpy as np

# Load the model and set it to evaluation mode
model_path = '/content/drive/MyDrive/Badminton_Player_Analysis_Project/SoloShuttlePose/draft/HitDataset/hitdetect.pth'
model = torch.load(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

HitModel(
  (gru1): GRU(82, 64, batch_first=True, bidirectional=True)
  (gru2): GRU(128, 64, batch_first=True, bidirectional=True)
  (global_maxpool): MaxPool1d(kernel_size=12, stride=12, padding=0, dilation=1, ceil_mode=False)
  (dense): Linear(in_features=128, out_features=3, bias=True)
  (softmax): Softmax(dim=-1)
)

In [ ]:
# Load the data
data_path = "/content/drive/MyDrive/Badminton_Player_Analysis_Project/SoloShuttlePose/draft/HitDataset/valid/Viktor_AXELSEN_Jonatan_CHRISTIE_Malaysia_Open_2022_SemiFinals/rally_1-12.csv"
df = pd.read_csv(data_path, converters={"ball": eval, "top": eval, "bottom": eval, "court": eval, "net": eval})


In [ ]:
df.head()

,frame,top,bottom,court,net,ball,pos,type
0,22862,"[[1074.3895263671875, 481.76519775390625], [10...","[[941.7289428710938, 594.3465576171875], [913....","[[689, 581], [1235, 581], [629, 736], [1300, 7...","[[562, 538], [562, 736], [1368, 736], [1368, 5...","[941, 576]",None,None
1,22863,"[[1074.359130859375, 483.114013671875], [1078....","[[941.7648315429688, 594.4612426757812], [913....","[[689, 581], [1235, 581], [629, 736], [1300, 7...","[[562, 538], [562, 736], [1368, 736], [1368, 5...","[941, 576]",None,None
2,22864,"[[1074.984619140625, 481.29290771484375], [107...","[[941.2090454101562, 594.1100463867188], [913....","[[689, 581], [1235, 581], [629, 736], [1300, 7...","[[562, 538], [562, 736], [1368, 736], [1368, 5...","[941, 576]",None,None
3,22865,"[[1074.7972412109375, 481.18975830078125], [10...","[[942.6001586914062, 591.3973388671875], [913....","[[689, 581], [1235, 581], [629, 736], [1300, 7...","[[562, 538], [562, 736], [1368, 736], [1368, 5...","[941, 576]",None,None
4,22866,"[[1075.5689697265625, 480.97625732421875], [10...","[[940.70556640625, 593.9485473632812], [913.44...","[[689, 581], [1235, 581], [629, 736], [1300, 7...","[[562, 538], [562, 736], [1368, 736], [1368, 5...","[941, 576]",None,None


In [ ]:
num_consecutive_frames = 12  # Assuming this is defined somewhere
normalization = True  # Assuming this is needed

final_list = []
true_list = []

rows = len(df)
print('df rows: ', rows)

# Determine if padding is needed
remainder = rows % 12
if remainder > 0:
    num_to_pad = 12 - remainder
else:
    num_to_pad = 0
print('padding needs: ', remainder)

# Pad the dataframe if necessary
if num_to_pad > 0:
    last_row = df.iloc[-1]
    padding_data = np.tile(last_row.values, (num_to_pad, 1))
    padded_df = pd.DataFrame(padding_data, columns=df.columns)
    df = pd.concat([df, padded_df], axis=0)
    df = df.reset_index(drop=True)

small_dataset = df
probs_list=[]

for i in range(len(small_dataset)):

    # Won't do the rest (seeing num_consecutive_frames ahead)
    if i>=len(small_dataset)-num_consecutive_frames:
        break


    # Prepare data for prediction (current_frame, ..., current_frame+num_consecutive_frames)
    oridata = small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
    oridata=oridata.reset_index(drop=True)
    data=[]
    if str(oridata.loc[0,'pos'])=='nan':
        true_list.append(0)
    elif str(oridata.loc[0,'pos'])=='top':
        true_list.append(1)
    elif str(oridata.loc[0,'pos'])=='bottom':
        true_list.append(2)
    else:
        true_list.append(0)

    for index,row in oridata.iterrows():
        top=np.array(row['top']).reshape(-1,2)
        bottom=np.array(row['bottom']).reshape(-1,2)
        court=np.array(row['court']).reshape(-1,2)
        # net=np.array(row['net']).reshape(-1,2)
        ball=np.array(row['ball']).reshape(-1,2)

        frame_data = np.concatenate((top, bottom, court, ball), axis=0)

        if normalization:
            # 将 x 坐标从 0 到 1920 映射到 1 到 2
            frame_data[:,0] /=1920

            # 将 y 坐标从 0 到 1080 映射到 1 到 2
            frame_data[:,1] /=1080

        data.append(frame_data.reshape(1,-1))

    # Predict the outcome
    data = np.array(data).reshape(1,-1)

    with torch.no_grad():
      outputs = model(torch.FloatTensor(data).to(device))

    pred = torch.argmax(outputs).item()
    probs = outputs[pred].item()
    final_list.append(pred)

print(true_list)
print(final_list)

df rows:  576
padding needs:  0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
def optimize_final_list_corrected(final_list, fps=30, num_consecutive_frames=5):
    optimized_list = final_list.copy()  # Work on a copy of the list to keep original intact

    # Helper function to apply the optimization rule
    def apply_optimization(start, end):
        for j in range(start, end - 1):  # Change all but the last to 0
            optimized_list[j] = 0

    i = 0
    while i < len(final_list):
        if final_list[i] in [1, 2]:
            start = i
            while i + 1 < len(final_list) and final_list[i + 1] == final_list[start]:
                i += 1
            end = i + 1
            if end - start >= num_consecutive_frames:  # If sequence is long enough, apply optimization
                apply_optimization(start, end)
        i += 1

    # Now enforce rally hit rules
    last_hit = None
    for i in range(1, len(optimized_list)):
        if optimized_list[i] in [1, 2]:
            # The paper said 0.5*fps but for the net drop i think it can be less than that so, i use 0.3 instead
            if last_hit is not None and (optimized_list[i] == last_hit or i - last_hit_index <= fps * 0.3):
                optimized_list[i] = 0  # Rule violation: too close or same player
            else:
                last_hit = optimized_list[i]
                last_hit_index = i

    return optimized_list

# Assuming final_list is as before
optimized_list = optimize_final_list_corrected(final_list)
print(true_list)
print(final_list)
print(optimized_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 

## Evaluation

In [ ]:
def optimize_final_list_corrected(final_list, fps=30, num_consecutive_frames=5):
    optimized_list = final_list.copy()  # Work on a copy of the list to keep original intact

    # Helper function to apply the optimization rule
    def apply_optimization(start, end):
        for j in range(start, end - 1):  # Change all but the last to 0
            optimized_list[j] = 0

    i = 0
    while i < len(final_list):
        if final_list[i] in [1, 2]:
            start = i
            while i + 1 < len(final_list) and final_list[i + 1] == final_list[start]:
                i += 1
            end = i + 1
            if end - start >= num_consecutive_frames:  # If sequence is long enough, apply optimization
                apply_optimization(start, end)
        i += 1

    # Now enforce rally hit rules
    last_hit = None
    for i in range(1, len(optimized_list)):
        if optimized_list[i] in [1, 2]:
            # The paper said 0.5*fps but for the net drop i think it can be less than that so, i use 0.3 instead
            if last_hit is not None and (optimized_list[i] == last_hit or i - last_hit_index <= fps * 0.3):
                optimized_list[i] = 0  # Rule violation: too close or same player
            else:
                last_hit = optimized_list[i]
                last_hit_index = i

    return optimized_list

In [ ]:
import glob
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

before_metrics = {
    "accuracy": [],
    "f1": [],
    "precision": [],
    "recall": []
}

after_metrics = {
    "accuracy": [],
    "f1": [],
    "precision": [],
    "recall": []
}

spans_metrics = {
    "+-1": {
        "accuracy": [],
        "f1": [],
        "precision": [],
        "recall": []
    },
    "+-3": {
        "accuracy": [],
        "f1": [],
        "precision": [],
        "recall": []
    }
}

def evaluate_with_tolerance(true_labels, predicted_labels, tolerance=0):
    # Adjusted lists for predictions within tolerance
    adjusted_predictions = []

    for i, true_label in enumerate(true_labels):
        # Initialize as incorrect prediction
        correct_within_tolerance = False

        # Check predictions within the tolerance range
        for offset in range(-tolerance, tolerance + 1):
            if (i + offset) >= 0 and (i + offset) < len(predicted_labels):
                if predicted_labels[i + offset] == true_label:
                    correct_within_tolerance = True
                    break

        adjusted_predictions.append(true_label if correct_within_tolerance else -1)  # Use -1 for incorrect predictions

    # Evaluate the adjusted predictions
    accuracy = accuracy_score(true_labels, adjusted_predictions, normalize=False) / len(true_labels)
    f1 = f1_score(true_labels, adjusted_predictions, average='macro', zero_division=0)
    precision = precision_score(true_labels, adjusted_predictions, average='macro', zero_division=0)
    recall = recall_score(true_labels, adjusted_predictions, average='macro', zero_division=0)

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

valid_dir = '/content/drive/MyDrive/Badminton_Player_Analysis_Project/SoloShuttlePose/draft/HitDataset/valid/'
for data_path in glob.glob(os.path.join(valid_dir, '*/*')):
    df = pd.read_csv(data_path, converters={"ball": eval, "top": eval, "bottom": eval, "court": eval, "net": eval})

    num_consecutive_frames = 12  # Assuming this is defined somewhere
    normalization = True  # Assuming this is needed

    final_list = []
    true_list = []

    rows = len(df)
    print('df rows: ', rows)

    # Determine if padding is needed
    remainder = rows % 12
    if remainder > 0:
        num_to_pad = 12 - remainder
    else:
        num_to_pad = 0
    print('padding needs: ', remainder)

    # Pad the dataframe if necessary
    if num_to_pad > 0:
        last_row = df.iloc[-1]
        padding_data = np.tile(last_row.values, (num_to_pad, 1))
        padded_df = pd.DataFrame(padding_data, columns=df.columns)
        df = pd.concat([df, padded_df], axis=0)
        df = df.reset_index(drop=True)

    small_dataset = df
    probs_list=[]

    for i in range(len(small_dataset)):

        # Won't do the rest (seeing num_consecutive_frames ahead)
        if i>=len(small_dataset)-num_consecutive_frames:
            break


        # Prepare data for prediction (current_frame, ..., current_frame+num_consecutive_frames)
        oridata = small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
        oridata=oridata.reset_index(drop=True)
        data=[]
        if str(oridata.loc[0,'pos'])=='nan':
            true_list.append(0)
        elif str(oridata.loc[0,'pos'])=='top':
            true_list.append(1)
        elif str(oridata.loc[0,'pos'])=='bottom':
            true_list.append(2)
        else:
            true_list.append(0)

        for index,row in oridata.iterrows():
            top=np.array(row['top']).reshape(-1,2)
            bottom=np.array(row['bottom']).reshape(-1,2)
            court=np.array(row['court']).reshape(-1,2)
            # net=np.array(row['net']).reshape(-1,2)
            ball=np.array(row['ball']).reshape(-1,2)

            frame_data = np.concatenate((top, bottom, court, ball), axis=0)

            if normalization:
                # 将 x 坐标从 0 到 1920 映射到 1 到 2
                frame_data[:,0] /=1920

                # 将 y 坐标从 0 到 1080 映射到 1 到 2
                frame_data[:,1] /=1080

            data.append(frame_data.reshape(1,-1))

        # Predict the outcome
        data = np.array(data).reshape(1,-1)

        with torch.no_grad():
          outputs = model(torch.FloatTensor(data).to(device))

        pred = torch.argmax(outputs).item()
        probs = outputs[pred].item()
        final_list.append(pred)

    before_metrics["accuracy"].append(accuracy_score(true_list, final_list))
    before_metrics["f1"].append(f1_score(true_list, final_list, average='macro'))
    before_metrics["precision"].append(precision_score(true_list, final_list, average='macro'))
    before_metrics["recall"].append(recall_score(true_list, final_list, average='macro'))

    # optimize
    optimized_list = optimize_final_list_corrected(final_list)

    after_metrics["accuracy"].append(accuracy_score(true_list, optimized_list))
    after_metrics["f1"].append(f1_score(true_list, optimized_list, average='macro'))
    after_metrics["precision"].append(precision_score(true_list, optimized_list, average='macro'))
    after_metrics["recall"].append(recall_score(true_list, optimized_list, average='macro'))

    # tolerance

    eval_result = evaluate_with_tolerance(true_list, optimized_list, tolerance=1)
    spans_metrics["+-1"]["accuracy"].append(eval_result["accuracy"])
    spans_metrics["+-1"]["f1"].append(eval_result["f1"])
    spans_metrics["+-1"]["precision"].append(eval_result["precision"])
    spans_metrics["+-1"]["recall"].append(eval_result["recall"])

    eval_result = evaluate_with_tolerance(true_list, optimized_list, tolerance=3)
    spans_metrics["+-3"]["accuracy"].append(eval_result["accuracy"])
    spans_metrics["+-3"]["f1"].append(eval_result["f1"])
    spans_metrics["+-3"]["precision"].append(eval_result["precision"])
    spans_metrics["+-3"]["recall"].append(eval_result["recall"])

df rows:  144
padding needs:  0
df rows:  174
padding needs:  6
df rows:  164
padding needs:  8
df rows:  306
padding needs:  6
df rows:  318
padding needs:  6
df rows:  190
padding needs:  10
df rows:  33
padding needs:  9
df rows:  392
padding needs:  8
df rows:  167
padding needs:  11
df rows:  601
padding needs:  1
df rows:  331
padding needs:  7
df rows:  274
padding needs:  10
df rows:  257
padding needs:  5
df rows:  173
padding needs:  5
df rows:  427
padding needs:  7
df rows:  117
padding needs:  9
df rows:  439
padding needs:  7
df rows:  135
padding needs:  3
df rows:  720
padding needs:  0
df rows:  411
padding needs:  3
df rows:  158
padding needs:  2
df rows:  203
padding needs:  11
df rows:  416
padding needs:  8
df rows:  91
padding needs:  7
df rows:  301
padding needs:  1
df rows:  67
padding needs:  7
df rows:  193
padding needs:  1
df rows:  85
padding needs:  1
df rows:  254
padding needs:  2
df rows:  223
padding needs:  7
df rows:  624
padding needs:  0
df rows:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


df rows:  46
padding needs:  10
df rows:  354
padding needs:  6
df rows:  254
padding needs:  2
df rows:  520
padding needs:  4
df rows:  631
padding needs:  7
df rows:  42
padding needs:  6
df rows:  191
padding needs:  11
df rows:  103
padding needs:  7
df rows:  523
padding needs:  7
df rows:  79
padding needs:  7
df rows:  46
padding needs:  10


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


df rows:  70
padding needs:  10
df rows:  220
padding needs:  4
df rows:  421
padding needs:  1
df rows:  176
padding needs:  8
df rows:  44
padding needs:  8
df rows:  125
padding needs:  5
df rows:  171
padding needs:  3
df rows:  199
padding needs:  7
df rows:  174
padding needs:  6
df rows:  175
padding needs:  7
df rows:  175
padding needs:  7
df rows:  664
padding needs:  4
df rows:  151
padding needs:  7
df rows:  379
padding needs:  7
df rows:  353
padding needs:  5
df rows:  187
padding needs:  7
df rows:  80
padding needs:  8
df rows:  630
padding needs:  6
df rows:  197
padding needs:  5
df rows:  371
padding needs:  11
df rows:  500
padding needs:  8
df rows:  111
padding needs:  3
df rows:  549
padding needs:  9
df rows:  86
padding needs:  2
df rows:  232
padding needs:  4
df rows:  571
padding needs:  7
df rows:  60
padding needs:  0
df rows:  143
padding needs:  11
df rows:  334
padding needs:  10
df rows:  206
padding needs:  2
df rows:  79
padding needs:  7
df rows:  

In [ ]:
print("Before optimization")
print("Accuracy: ", sum(before_metrics["accuracy"])/len(before_metrics["accuracy"]))
print("F1: ", sum(before_metrics["f1"])/len(before_metrics["f1"]))
print("Precision: ", sum(before_metrics["precision"])/len(before_metrics["precision"]))
print("Recall: ", sum(before_metrics["recall"])/len(before_metrics["recall"]))
print('-'*40)
print("After optimization")
print("Accuracy: ", sum(after_metrics["accuracy"])/len(after_metrics["accuracy"]))
print("F1: ", sum(after_metrics["f1"])/len(after_metrics["f1"]))
print("Precision: ", sum(after_metrics["precision"])/len(after_metrics["precision"]))
print("Recall: ", sum(after_metrics["recall"])/len(after_metrics["recall"]))
print('-'*40)
print("After optimization (tolerance=1)")
print("Accuracy: ", sum(spans_metrics["+-1"]["accuracy"])/len(spans_metrics["+-1"]["accuracy"]))
print("F1: ", sum(spans_metrics["+-1"]["f1"])/len(spans_metrics["+-1"]["f1"]))
print("Precision: ", sum(spans_metrics["+-1"]["precision"])/len(spans_metrics["+-1"]["precision"]))
print("Recall: ", sum(spans_metrics["+-1"]["recall"])/len(spans_metrics["+-1"]["recall"]))
print('-'*40)
print("After Before optimization (tolerance=3)")
print("Accuracy: ", sum(spans_metrics["+-3"]["accuracy"])/len(spans_metrics["+-3"]["accuracy"]))
print("F1: ", sum(spans_metrics["+-3"]["f1"])/len(spans_metrics["+-3"]["f1"]))
print("Precision: ", sum(spans_metrics["+-3"]["precision"])/len(spans_metrics["+-3"]["precision"]))
print("Recall: ", sum(spans_metrics["+-3"]["recall"])/len(spans_metrics["+-3"]["recall"]))
print('-'*40)

Before optimization
Accuracy:  0.5561834409420404
F1:  0.3151853870134976
Precision:  0.37412942344700545
Recall:  0.676569489284776
----------------------------------------
After optimization
Accuracy:  0.9371324939327234
F1:  0.4687846012245381
Precision:  0.4688617406629856
Recall:  0.4709861403042159
----------------------------------------
After optimization (tolerance=1)
Accuracy:  0.9793402921120599
F1:  0.5702837060980094
Precision:  0.6543478260869565
Recall:  0.5258901417154757
----------------------------------------
After Before optimization (tolerance=3)
Accuracy:  0.98857201594725
F1:  0.6966113640352827
Precision:  0.7347826086956522
Recall:  0.6723657384814673
----------------------------------------
